<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/5_combined_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-16 18:56:13--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11733 (11K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.46K  --.-KB/s    in 0s      

2022-06-16 18:56:14 (28.0 MB/s) - ‘utils.py’ saved [11733/11733]



In [2]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, get_class_weight

In [2]:
BATCH_SIZE = 32
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16
HUB_URL = 'https://tfhub.dev/google/bit/m-r50x1/1'

In [3]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [4]:
train_ds, _ = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [5]:
classes, class_weight = get_class_weight()

In [6]:
train_ds = (
    train_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [ ]:
rgb_input = keras.Input(shape=IMG_SIZE+(3,))
hub_module = hub.KerasLayer(HUB_URL)

rgb_features = hub_module(rgb_input)

xyz_input = tfio.experimental.color.rgb_to_xyz(rgb_input)
xyz_features = hub_module(xyz_input)

hsv_input = tf.image.rgb_to_hsv(rgb_input)
hsv_features = hub_module(hsv_input)

features = keras.layers.Concatenate()([rgb_features, xyz_features, hsv_features])

output = keras.layers.Dense(N_CLASSES, activation='softmax')(features)

model = keras.models.Model(inputs=[rgb_input], outputs=[output])

In [17]:
model._name = 'combined-model'

In [18]:
metrics = [
            keras.metrics.AUC(name='auc', curve='PR', num_thresholds=100),
            'accuracy'
          ]

loss_fn = keras.losses.CategoricalCrossentropy()
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(1e-2, 533, .9)

model.compile(loss=loss_fn,
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=metrics)

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
                                              monitor='auc', 
                                              verbose=1,
                                              patience=3,
                                              restore_best_weights=True,
                                              mode='max')

check_point_path = os.path.join('./logs/checkpoints', model._name+"-{epoch:02d}.h5")
check_point = keras.callbacks.ModelCheckpoint(
                                            filepath=check_point_path,
                                            monitor='auc',
                                            save_best_only=True,
                                            mode='max')
        
callbacks = [early_stopping, check_point]

In [ ]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=20,
                    class_weight=class_weight)

Epoch 1/20
519/519 [==============================] - 1183s 2s/step - loss: 416.2709 - auc: 0.5196 - accuracy: 0.6564
Epoch 2/20
519/519 [==============================] - 421s 808ms/step - loss: 193.5110 - auc: 0.6601 - accuracy: 0.7668
Epoch 3/20
519/519 [==============================] - 420s 807ms/step - loss: 183.8564 - auc: 0.6948 - accuracy: 0.7963
Epoch 4/20
519/519 [==============================] - 420s 808ms/step - loss: 150.1589 - auc: 0.7323 - accuracy: 0.8227
Epoch 5/20
519/519 [==============================] - 420s 807ms/step - loss: 109.7715 - auc: 0.7796 - accuracy: 0.8523
Epoch 6/20
519/519 [==============================] - 420s 807ms/step - loss: 91.0485 - auc: 0.8038 - accuracy: 0.8718
Epoch 7/20
519/519 [==============================] - 420s 807ms/step - loss: 84.0862 - auc: 0.8191 - accuracy: 0.8809
Epoch 8/20
519/519 [==============================] - 420s 806ms/step - loss: 65.2560 - auc: 0.8404 - accuracy: 0.8941
Epoch 9/20
519/519 [=========================

In [ ]:
!cp ./logs/checkpoints/combined-model-10.h5 ./drive/MyDrive/checkpoints/combined-model-10.2.h5
!cp ./logs/checkpoints/combined-model-15.h5 ./drive/MyDrive/checkpoints/combined-model-15.2.h5
!cp ./logs/checkpoints/combined-model-20.h5 ./drive/MyDrive/checkpoints/combined-model-20.2.h5

In [8]:
model = keras.models.load_model("./drive/MyDrive/checkpoints/combined-model-20.2.h5",
                                custom_objects={'KerasLayer':hub.KerasLayer})

In [10]:
history = model.fit(train_ds,
                    callbacks=callbacks,
                    epochs=30,
                    initial_epoch=20,
                    class_weight=class_weight)

Epoch 21/30
519/519 [==============================] - 2015s 4s/step - loss: 4.9946 - auc: 0.9739 - accuracy: 0.9785
Epoch 22/30
519/519 [==============================] - 424s 814ms/step - loss: 4.6762 - auc: 0.9754 - accuracy: 0.9794
Epoch 23/30
519/519 [==============================] - 424s 814ms/step - loss: 3.8836 - auc: 0.9779 - accuracy: 0.9818
Epoch 24/30
519/519 [==============================] - 424s 814ms/step - loss: 2.7520 - auc: 0.9840 - accuracy: 0.9850
Epoch 25/30
519/519 [==============================] - 423s 813ms/step - loss: 3.5704 - auc: 0.9797 - accuracy: 0.9826
Epoch 26/30
519/519 [==============================] - 423s 814ms/step - loss: 2.4298 - auc: 0.9860 - accuracy: 0.9881
Epoch 27/30
519/519 [==============================] - 424s 815ms/step - loss: 2.0797 - auc: 0.9873 - accuracy: 0.9892
Epoch 28/30
519/519 [==============================] - 424s 814ms/step - loss: 2.0638 - auc: 0.9892 - accuracy: 0.9891
Epoch 29/30
519/519 [=============================

In [12]:
!cp ./logs/checkpoints/combined-model-24.h5 ./drive/MyDrive/checkpoints/combined-model-24.2.h5
!cp ./logs/checkpoints/combined-model-29.h5 ./drive/MyDrive/checkpoints/combined-model-29.2.h5